# Overview
* Install haptools to process the vcf file (https://haptools.readthedocs.io/en/stable/index.html)
* Get the vcf data from 1000 Genomes (In our project we will be using chr 21)
* Convert VCF file (.vcf.gz) to a hap file (.hap)
* Simulate phenotypes

# Installing haptools 

In [6]:
# Allows us to run haptools in your directory
pip install haptools
ls ~/.local/bin/
export PATH=$PATH:$HOME/.local/bin

# Retriving the Real World Data

Chromosome 21 Real Data For Genotypes (1000 Genomes phase 1 release)
ftp://ftp.1000genomes.ebi.ac.uk/vol1/ftp/phase1/analysis_results/integrated_call_sets/ALL.chr21.integrated_phase1_v3.20101123.snps_indels_svs.genotypes.vcf.gz

In [ ]:
# wget [insert .vcf.gz file here]
wget ftp://ftp.1000genomes.ebi.ac.uk/vol1/ftp/phase1/analysis_results/integrated_call_sets/ALL.chr21.integrated_phase1_v3.20101123.snps_indels_svs.genotypes.vcf.gz
    
# zcat [insert .vcf.gz file here] | less -S
zcat ALL.chr21.integrated_phase1_v3.20101123.snps_indels_svs.genotypes.vcf.gz | less -S

# "ctrl+v" to scroll through the page to look for variants you are interested in

# Converting .vcf files to .hap

The code below was used from the haptools documentation: https://haptools.readthedocs.io/en/stable/api/examples.html#api-examples-snps2hap (given permission to use this pre-written code)

In [7]:
from haptools import data
from haptools.sim_phenotype import Haplotype

# which variants do we want to write to the haplotype file?
variants = {"rs149635655", "rs141306699"}

# load the genotypes file
# you can use either a VCF or PGEN file
gt = data.GenotypesVCF("ALL.chr21.integrated_phase1_v3.20101123.snps_indels_svs.genotypes.vcf.gz")
gt.read(variants=variants)

# initialize an empty haplotype file
hp = data.Haplotypes("output.hap", haplotype=Haplotype)
hp.data = {}

for variant in gt.variants:
    ID, chrom, pos, alleles = variant[["id", "chrom", "pos", "alleles"]]
    end = pos + len(alleles[1])

    # create a haplotype line in the .hap file
    # you should fill out "beta" with your own value
    hp.data[ID] = Haplotype(chrom=chrom, start=pos, end=end, id=ID, beta=0.5)

    # create variant lines for each haplotype
    hp.data[ID].variants = (data.Variant(start=pos, end=end, id=ID, allele=alleles[1]),)

hp.write()

# Use haptools to simulate phenotypes

haptools simphenotype \
--replications INT \
--heritability FLOAT \
--prevalence FLOAT \
--region TEXT \
--sample SAMPLE --sample SAMPLE \
--samples-file FILENAME \
--id ID --id ID \
--ids-file FILENAME \
--chunk-size INT \
--output PATH \
--verbosity [CRITICAL|ERROR|WARNING|INFO|DEBUG|NOTSET] \
GENOTYPES HAPLOTYPES

In [ ]:
# Index the .vcf.gz file using tabix so we can retrieve information from the file faster
tabix -p vcf ALL.chr21.integrated_phase1_v3.20101123.snps_indels_svs.genotypes.vcf.gz

# Uses the haptools function simphenotype to simulate phenotypes for our genotypes so we can run GWAS on it
haptools simphenotype ALL.chr21.integrated_phase1_v3.20101123.snps_indels_svs.genotypes.vcf.gz output.hap -o simulated.pheno